
We start off with importing the necessary packages (Pandas and NumPy), and our self-written SPCA script which contains all the code for performing the SPCA estimation.

In [1]:
import pandas as pd
import numpy as np
import spca
import utils

We will start by looking at the pitprops dataset, thus using pandas we import the dataset.

In [2]:
# Import both the pitprops and gene datasets; for the pitprops dataset, use the first column as the indices.
pitprops = pd.read_csv("data/pitprops.csv", index_col=0)

We set the <img style="transform: translateY(0.1em); background: white;" src="https://render.githubusercontent.com/render/math?math=l_2">-norm equal to the <img style="transform: translateY(0.1em); background: white;" src="https://render.githubusercontent.com/render/math?math=%5Clambda">'s which are provided by Zou, Hastie, and Tibshirani (2006). We then run our self-written function sparcepca() to do the actual SPCA analysis. We first look at our self-written function which uses the elasticnet function from the scikit-learn package.

In [3]:
lambda2 = np.array([0.06, 0.16, 0.1, 0.5, 0.5, 0.5])
A, B, vh = spca.sparcepca(X=pitprops, lambda2=lambda2, lambda1=0.1, k=6, max_iteration=int(1e3), threshold=1e-4, type="cov")
print(B)
print(utils.variance(pitprops, vh))
print(utils.nonzeroloadings(B))

[[ 0.33098437  0.32997669  0.06424873  0.13476448  0.00933825  0.22714218
   0.34982834  0.23142629  0.28760249  0.31702146 -0.         -0.06014143
  -0.05975811]
 [ 0.05790705  0.0364325   0.27544877  0.20307331 -0.04056238 -0.
  -0.0470065  -0.03953152  0.         -0.08423594  0.05903374  0.14876262
   0.13451519]
 [ 0.08314083  0.1080429  -0.02490257 -0.18616354 -0.25928798 -0.26718444
  -0.0690874   0.11981492  0.08803495  0.01821092  0.         -0.00678254
   0.15120056]
 [ 0.          0.         -0.         -0.         -0.          0.
   0.         -0.         -0.          0.         -0.03526534  0.
   0.        ]
 [ 0.          0.         -0.         -0.         -0.          0.
   0.         -0.          0.         -0.          0.          0.
  -0.        ]
 [-0.         -0.          0.          0.         -0.         -0.
  -0.          0.         -0.          0.         -0.          0.
  -0.        ]]
[[ 0.32625641  0.32523184  0.06034913  0.13460588  0.0171565   0.23412735
   

Second, we look at our self written sparcepca() function using our self-written elasticnet minimizer.

In [ ]:
A, B, vh = spca.sparcepca(X=pitprops, lambda2=lambda2, lambda1=0.1, k=6, max_iteration=int(1e3), threshold=1e-4, type="cov", optimizer="self")
print(B)

To-do w.r.t. pitprops
- Convert sparcepca() output into an easy-to-copy table
- Compare self-built function to sota

Then w.r.t. the gene dataset;
- Adapt spca and elasticnet functions to gene dataset
- Output/plot outcomes from gene dataset like in Zou, Hastie, and Tibshirani (2006)
- Again, compare self-built vs. sota

Second, we will take a look at the gene dataset. We start here with some data cleanup; remove present/absent calls which are not relevant for our research, take subsample of 144 and set row names to "Accession" column (Note the description column is not unique, while the accession column is).

In [5]:
genedata = pd.read_csv("data/GCM_Total.csv")
genedata = genedata[genedata.columns.drop(list(genedata.filter(regex='Unnamed')))]
genedata.index = genedata.iloc[:, 1]
genedata = genedata.iloc[:, 2:146]

In [6]:
lambda2 = np.random.rand(16000,1)

In [8]:
print(spca.sparcepca(X=genedata, lambda2=lambda2, lambda1=0.1, k=140, max_iteration=int(1e3), threshold=1e-2, type="data"))

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.795e+04, tolerance: 8.221e+04
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.423e+05, tolerance: 7.118e+04
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.332e+05, tolerance: 6.805e+04

[[ 0.01465199  0.01196532  0.06658201 ... -0.02624264  0.0846397
   0.05241951]
 [ 0.04299049  0.02124662  0.05980042 ...  0.02885238  0.08330536
   0.09593805]
 [-0.02361147 -0.01310623  0.04269185 ...  0.02328669  0.11184929
   0.01769022]
 ...
 [-0.03650766  0.17035877 -0.0009751  ...  0.22777007 -0.00752594
   0.00476353]
 [-0.08066395  0.11345775  0.06085784 ...  0.1798375  -0.01748256
  -0.00748418]
 [-0.00790522 -0.10503504  0.0138789  ...  0.08226862 -0.01734887
  -0.00463578]]


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.937e+05, tolerance: 2.022e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.951e+05, tolerance: 1.324e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.061e+05, tolerance: 1.256e+05

[[ 0.01466294  0.01201436  0.06657762 ... -0.02626611  0.08464059
   0.05242749]
 [ 0.04224672  0.02064801  0.06074954 ...  0.02977454  0.08535454
   0.09764749]
 [-0.02380135 -0.0135339   0.04258021 ...  0.02275719  0.11002827
   0.01782146]
 ...
 [-0.00148576 -0.02716145 -0.04534802 ...  0.10470224 -0.00461227
  -0.00839064]
 [-0.05425456  0.15331714  0.05492327 ...  0.16838181 -0.00937554
  -0.00959174]
 [-0.03128712 -0.00398131  0.03529877 ... -0.02036216  0.01042086
   0.00143238]]


KeyboardInterrupt: 